In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D, Utilities as U, Parsers as P
import matplotlib.pyplot as plt
import numpy as np

In [3]:
omp = 1
mpi_run = 'mpirun -np 8'

# DFT analysis of GaAs

This notebook contains the analysis of the GaAs performed with QuantumESPRESSO for the construction
of the band structure used in the TD analysis.

## Construction of the GS

The GS is built using the _converged_ parameters previously obtained, in particular

In [4]:
ecut_conv = 80
k_conv = 6
lat_eq = 10.475

In [5]:
run_dir = 'GS'
name = 'GS_input'

In [6]:
inp = I.PwInput()
inp.set_scf(force_symmorphic=True,conv_thr=1e-6)
inp.set_prefix(name)
inp.set_pseudo_dir(pseudo_dir='../pseudos')
inp.set_lattice(2,celldm1=lat_eq)
inp.set_occupations(occupations='fixed')
inp.add_atom('Ga','Ga_rel.pz-rrkj3.UPF')
inp.add_atom('As','As_rel.pz-rrkj3.UPF')
inp.set_atoms_number(2)
inp.set_spinorbit()
inp.set_atomic_positions([['Ga',[0.,0.,0.]],['As',[0.25,0.25,0.25]]],type='alat')
inp.set_energy_cutoff(ecut_conv)
inp.set_kpoints(points=[k_conv,k_conv,k_conv])
inp

{'control': {'calculation': "'scf'",
  'verbosity': "'high'",
  'prefix': "'GS_input'",
  'pseudo_dir': "'../pseudos'"},
 'system': {'force_symmorphic': '.true.',
  'ibrav': 2,
  'celldm(1)': 10.475,
  'occupations': "'fixed'",
  'ntyp': '2',
  'nat': '2',
  'lspinorb': '.true.',
  'noncolin': '.true.',
  'ecutwfc': 80},
 'electrons': {'conv_thr': 1e-06, 'diago_full_acc': '.false.'},
 'ions': {},
 'cell': {},
 'atomic_species': {'Ga': ['1.0', 'Ga_rel.pz-rrkj3.UPF'],
  'As': ['1.0', 'As_rel.pz-rrkj3.UPF']},
 'atomic_positions': {'type': 'alat',
  'values': [['Ga', [0.0, 0.0, 0.0]], ['As', [0.25, 0.25, 0.25]]]},
 'kpoints': {'type': 'automatic', 'values': ([6, 6, 6], [0.0, 0.0, 0.0])},
 'cell_parameters': {}}

In [7]:
code = C.QeCalculator(omp=omp,mpi_run=mpi_run,skip=True)

Initialize a parallel QuantumESPRESSO calculator with scheduler direct


In [8]:
result = code.run(inputs=[inp],names=['GS_input'],run_dir=run_dir)

Skip the computation for input GS_input
Job completed


## Calculation of empty bands

We build the empty bands associated to specific $k$ points. In particular

In [9]:
kmax = [[-0.0247738, 0.00762269, 0.02365302,1.0]],4.36  # Correspond to the fastest cc' oscillation of period 4.36 ps
kmed = [[-0.01655783, 0.00728803, 0.02452443,1.0]],6.86 # Correspond to the a 'medium' cc' oscillation with 0.602 meV
klist = [kmax,kmed]

In [10]:
run_dir = 'NSCF'
source_dir = 'GS_convergence/gs_converged.save'
num_bands = 16

In [11]:
inp.set_nscf(num_bands,force_symmorphic=True,conv_thr=1e-6)

In [12]:
nscf = D.Dataset(run_dir=run_dir)

In [13]:
for k in klist:
    idd = {'bnds' : num_bands, 'T' : k[1]}
    inp.set_kpoints(type='tpiba',klist=k[0])
    inp.set_prefix(D.name_from_id(idd))
    nscf.append_run(id=idd,input=inp,runner=code,source_dir = 'GS/GS_input.save')

In [14]:
nscf.set_postprocessing_function(D.QE_parse_data)

In [15]:
results = nscf.run()

The folder NSCF/T_4.36-bnds_16.save already exsists. Source folder GS/GS_input.save not copied
The folder NSCF/T_6.86-bnds_16.save already exsists. Source folder GS/GS_input.save not copied
Skip the computation for input T_4.36-bnds_16
Skip the computation for input T_6.86-bnds_16
Job completed


We compute the period associated to the $cc'$ transition for the two cases

In [16]:
demax = results[0].get_transitions(initial=[8,9],final=[8,9])
demed = results[1].get_transitions(initial=[8,9],final=[8,9])

In [17]:
1e3*demax[0][0]

0.9491628167177524

In [18]:
U.Planck_ev_ps/demax[0][0]

4.357174157223441

In [19]:
U.Planck_ev_ps/demed[0][0]

6.857733945082544

both expressed in ps.

We use these bands to study the oscillation of the degree of spin polarization when 
the system is subject to a laser pump. We compute the energy of the $c \rightarrow v$
transitions assciated to the light and heavy holes. We add the scissor previously
computed (the set gap cannot be used since the $\Gamma$ point is not present
in the $k$-points list)

In [20]:
scissor = 0.545
tcv_max = results[0].get_transitions(initial=[4,5,6,7],final=[8,9])[0] + scissor
tcv_med = results[1].get_transitions(initial=[4,5,6,7],final=[8,9])[0] + scissor

In [21]:
tcv_max

array([1.60788224, 1.6088314 , 1.60528452, 1.60623368, 1.53077349,
       1.53172265, 1.53059458, 1.53154374])

In [22]:
tcv_med

array([1.56638245, 1.56698552, 1.56482034, 1.56542341, 1.50650793,
       1.50711099, 1.50639341, 1.50699647])

In what follows we decide to set
* $\omega_{pump}$ = 1.531 for kmax
* $\omega_{pump}$ = 1.506 for kmed

so that the pump is resonant with transitions from highest valence bands (6,7) 
to the conduction bands (8,9).